In [18]:
from chess import *
import chess
import chess.pgn
from stockfish import Stockfish
import random
import numpy as np
import csv
import pandas as pd


In [19]:
def encode_board(board):
    """
    Function that encodes a board into a 14x8x8 tensor
    """
    # Initialize 14x8x8 matrix with all zeros
    encoded_board = np.zeros((14, 8, 8), dtype=np.int8)

    # Encode the pieces on the board
    piece_map = {
        chess.PAWN: 0,
        chess.KNIGHT: 1,
        chess.BISHOP: 2,
        chess.ROOK: 3,
        chess.QUEEN: 4,
        chess.KING: 5
    }
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            row = chess.square_rank(square)
            column = chess.square_file(square)
            color = int(not piece.color)
            value = piece_map[piece.piece_type]
            encoded_board[ color * 6 + value, ((8-row) % 8) - 1, column] = 1 #if piece.color == True else -1

    #Encode the possible moves for white and black
    aux = board.turn
    board.turn = chess.WHITE

    for move in board.legal_moves:
        #rank_from = chess.square_rank(move.from_square)
        #file_from = chess.square_file(move.from_square)
        rank_to = chess.square_rank(move.to_square)
        file_to = chess.square_file(move.to_square)
        encoded_board[12, ((8-rank_to) % 8) - 1, file_to] = 1

    board.turn = chess.BLACK
    for move in board.legal_moves:
        #rank_from = chess.square_rank(move.from_square)
        #file_from = chess.square_file(move.from_square)
        rank_to = chess.square_rank(move.to_square)
        file_to = chess.square_file(move.to_square)
        encoded_board[13, ((8-rank_to) % 8) - 1, file_to] = 1
            
    board.turn = aux    
    return encoded_board



In [20]:
#Manipulation of the csv
#Only use one time on a CSV

with open('chessData.csv', 'r') as file:

    # Read the CSV file into a pandas dataframe
    df = pd.read_csv(file)


In [21]:
# Remove rows containing a specific character
df = df[~df['Evaluation'].str.contains('#')]

# Open the CSV file in write mode and write the updated dataframe to the file
with open('chessData.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(df.columns)
    writer.writerows(df.values)

In [28]:
def database_creator(totalGames = 1000):
    """
    Creates a db of two columns, a board tensor, shape 14x8x8, and its evaluation, an integer
    Loops through a CSV file and stores it an npz file to be used in the future
    """
    i = 0

    a = np.zeros((totalGames, 14, 8, 8), dtype=np.int8)
    b = np.zeros((totalGames,), dtype=np.int32)

    
    with open('chessData.csv', 'r') as file:

        reader = csv.reader(file)

        for row in reader:

            if i != 0:
                fen = row[0]
                eval = int(row[1])

                board = chess.Board(fen)
                final_board = encode_board(board)
                
                a[i-1] = final_board
                b[i-1] = eval

            i += 1
                
            if i >= totalGames:
                break
        
        np.savez("eval.npz", boards = a, evaluations = b)

database_creator()
